## Read annotation from the BioLiP text file

Each line in the annotation file BioLiP_*.txt annotates for each ligand-protein interaction site in BioLiP.  We then search the proteins in Davis datasets for pocket residue info. Ref to NodeCoder `build_biolip_df` function. 

The columns are (from left to right):
- 01	PDB ID
- 02	Receptor chain
- 03	Resolution. "-1.00" stands for lack of resolution information, e.g. for NMR
- 04	Binding site number code
- 05	Ligand ID in the Chemical Component Dictionary (CCD) used by the PDB database
- 06	Ligand chain
- 07	Ligand serial number
- 08      Binding site residues (with PDB residue numbering)
- 09      Binding site residues (with residue re-numbered starting from 1)
- 10	Catalytic site residues (different sites are separated by ';') (with PDB residue numbering)
- 11      Catalytic site residues (different sites are separated by ';') (with residue re-numbered starting from 1)
- 12	EC number
- 13	GO terms
- 14	Binding affinity by manual survey of the original literature. The information in '()' is the PubMed ID
- 15	Binding affinity provided by the Binding MOAD database. The information in '()' is the ligand information in Binding MOAD
- 16	Binding affinity provided by the PDBbind-CN database. The information in '()' is the ligand information in PDBbind-CN
- 17	Binding affinity provided by the BindingDB database
- 18	UniProt ID
- 19	PubMed ID
- 20	Residue sequence number of the ligand (field _atom_site.auth_seq_id in PDBx/mmCIF format)
- 21	Receptor sequence

## Pocket Label From UniProt Sites File

- ✅Pocket Protein List with 229 labeled proteins.

In [1]:
import pickle
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
from torchdrug import utils

path = "../../../data/dta-datasets/KIBA/"
protein_file = "../../../data/dta-datasets/KIBA/kiba_proteins.csv"
protein_pkl = '../../../data/dta-datasets/KIBA/gearnetesm650m_Protein.pkl'

In [2]:
with utils.smart_open(protein_pkl, "rb") as fin:
    protein_list = pickle.load(fin)

In [3]:
len(protein_list)

229

In [4]:
protein_list

[PackedProtein(batch_size=1, num_atoms=[431], num_bonds=[7507], num_residues=[431]),
 PackedProtein(batch_size=1, num_atoms=[574], num_bonds=[8301], num_residues=[574]),
 PackedProtein(batch_size=1, num_atoms=[1044], num_bonds=[19549], num_residues=[1044]),
 PackedProtein(batch_size=1, num_atoms=[725], num_bonds=[11806], num_residues=[725]),
 PackedProtein(batch_size=1, num_atoms=[970], num_bonds=[13888], num_residues=[970]),
 PackedProtein(batch_size=1, num_atoms=[476], num_bonds=[8081], num_residues=[476]),
 PackedProtein(batch_size=1, num_atoms=[756], num_bonds=[12719], num_residues=[756]),
 PackedProtein(batch_size=1, num_atoms=[403], num_bonds=[6331], num_residues=[403]),
 PackedProtein(batch_size=1, num_atoms=[740], num_bonds=[11870], num_residues=[740]),
 PackedProtein(batch_size=1, num_atoms=[745], num_bonds=[12311], num_residues=[745]),
 PackedProtein(batch_size=1, num_atoms=[365], num_bonds=[6814], num_residues=[365]),
 PackedProtein(batch_size=1, num_atoms=[556], num_bonds=[

In [5]:
label_dict = json.load(open(f"{path}BioLiP/pocket_label.txt"))

In [6]:
total_nums = 0
for value in label_dict.values():
    total_nums += len(value)

In [7]:
total_nums

2987

In [8]:
label_dict['P42336']

[61, 238, 409, 606, 908]

In [9]:
protein_df = pd.read_csv(protein_file)
protein_df

,Gene,Sequence,AA length,PDBID
0,O00141,MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNS...,431,O00141
1,O00311,MEASLGIQMDEPMAFSPQRDRFQAEGSLKKNEQNFKLAGVKKDIEK...,574,O00311
2,O00329,MPPGVDCPMEFWTKEENQSVVVDFLLPTGVYLNFPVSRNANLSTIK...,1044,O00329
3,O00418,MADEDLIFRLEGVDGGQSPRAGHDGDSDGDSDDEEGYFICPITDDP...,725,O00418
4,O00444,MATCIGEKIEDFKVGNLLGKGSFAGVYRAESIHTGLEVAIKMIDKK...,970,O00444
...,...,...,...,...
224,Q9Y243,MSDVTIVKEGWVQKRGEYIKNWRPRYFLLKTDGSFIGYKEKPQDVD...,479,Q9Y243
225,Q9Y463,MAVPPGHGPFSGFPGPQEHTQVLPDVRLLPRRLPLAFRDATSAPLR...,629,Q9Y463
226,Q9Y478,MGNTSSERAALERHGGHKTPRRDSSGGTKDGDRPKILMDSPEDADL...,270,Q9Y478
227,Q9Y4K4,MEAPLRPAADILRRNPQQDYELVQRVGSGTYGDVYKARNVHTGELA...,846,Q9Y4K4


In [10]:
protein_df['PDBID'][0]

'O00141'

In [12]:
label_list = []
pocket_list = []
indexes = range(protein_df["PDBID"].shape[0])  # dataset_df["PDB_File"].shape[0]
indexes = tqdm(indexes, "Constructing Pocket from File: ")
for index in indexes:
    current_pdb = protein_df['PDBID'][index]
    pocket_key = label_dict[current_pdb]
    current_pocket = [x - 1 for x in set(pocket_key)]  # unique protein
    current_protein = protein_list[index]
    current_length = current_protein.num_residue.item()
    pocket = current_protein.residue_mask(index=current_pocket, compact=True)
    pocket_list.append(pocket)

    pocket_dict = {}
    pocket_dict['mask'] = np.ones(current_length, dtype='int32')  # all true
    pocket_dict['target'] = np.zeros(current_length, dtype='int32')  
    for pocket_index in current_pocket:
        pocket_dict['target'][pocket_index] = 1
    label_list.append(pocket_dict)

Constructing Pocket from File: 100%|██████████| 229/229 [00:00<00:00, 743.65it/s]


In [13]:
len(label_list)

229

In [14]:
len(pocket_list)

229

In [15]:
pocket_list

[PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[90], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[91], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[32], num_bonds=[171], num_residues=[32]),
 PackedProtein(batch_size=1, num_atoms=[7], num_bonds=[29], num_residues=[7]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[92], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[91], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[94], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[9], num_bonds=[29], num_residues=[9]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[88], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[94], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[92], num_residues=[11]),
 PackedProtein(batch_size=1, num_atoms=[11], num_bonds=[36], num_residues=[11]),
 PackedProtein(batch_size=1, nu

In [13]:
label_list[0]

{'mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1

In [14]:
label_pkl = '../../../data/dta-datasets/KIBA/gearnet_labeled.pkl'
with utils.smart_open(label_pkl, "wb") as fout:
    pickle.dump(label_list, fout)

In [16]:
pocket_pkl = '../../../data/dta-datasets/KIBA/gearnetesm_labeledpocket_Protein.pkl'
with utils.smart_open(pocket_pkl, "wb") as fout:
    pickle.dump(pocket_list, fout)

## Pocket Prediction Performance with P2Rank

Get the label

In [1]:
import torch
import pickle
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
from torchdrug import utils, metrics

path = "../../../data/dta-datasets/KIBA/"
protein_file = "../../../data/dta-datasets/KIBA/kiba_proteins.csv"

In [2]:
label_pkl = '../../../data/dta-datasets/KIBA/gearnet_labeled.pkl'
with utils.smart_open(label_pkl, "rb") as fin:
    label_list = pickle.load(fin)

In [3]:
len(label_list)

229

In [4]:
label_list[0]

{'mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1

Get the prediction from P2Rank

In [5]:
protein_df = pd.read_csv(protein_file, usecols=['Gene', 'PDBID'])
protein_df

,Gene,PDBID
0,O00141,O00141
1,O00311,O00311
2,O00329,O00329
3,O00418,O00418
4,O00444,O00444
...,...,...
224,Q9Y243,Q9Y243
225,Q9Y463,Q9Y463
226,Q9Y478,Q9Y478
227,Q9Y4K4,Q9Y4K4


In [8]:
auprc_list = []
auroc_list =[]
pdbs = protein_df["PDBID"].tolist() 
indexes = range(len(pdbs))
indexes = tqdm(indexes, "Cal the AUR and PRC from CSV: ")
for index in indexes:
    # get the label
    current_target = label_list[index]
    target = torch.as_tensor(current_target['target'], dtype=torch.long)
    mask = torch.as_tensor(current_target["mask"], dtype=torch.bool)
    labeled = ~torch.isnan(target) & mask
    # get the pred
    pdb_name = pdbs[index]
    csv_file = path + 'p2rank/predict_protein_list/AF-' + pdbs[index] + '-F1-model_v4.pdb_residues.csv'
    pdb_residues = pd.read_csv(csv_file)
    pdb_residues.columns = pdb_residues.columns.str.strip()
    pred_list = pdb_residues["probability"].tolist()
    pred = torch.as_tensor(pred_list, dtype=torch.float32)
    # if target.shape[0] != target.shape[0]:
    #     print("Error") 
    # get the result
    current_auprc = metrics.area_under_prc(pred[labeled], target[labeled])
    auprc_list.append(current_auprc)
    current_auroc = metrics.area_under_roc(pred[labeled], target[labeled])
    auroc_list.append(current_auroc)

Cal the AUR and PRC from CSV: 100%|██████████| 229/229 [00:00<00:00, 596.49it/s]


In [9]:
# get the mean result
AUPRC = sum(auprc_list) / len(auprc_list)
AUPRC

tensor(0.2290)

In [10]:
AUROC = sum(auroc_list) / len(auroc_list)
AUROC

tensor(0.8606)